<a href="https://colab.research.google.com/github/disha-ramchandani/P2P-loans/blob/main/Random_forest_ensemble_based_learner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [4]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [5]:
df=pd.read_csv('Final_project_dataset')
df.head()

,Unnamed: 0,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,0,22.0,59000.0,RENT,14.5,PERSONAL,D,35000,16.02,0.59,Y,3.0
1,1,21.0,9600.0,OWN,5.0,EDUCATION,B,1000,11.14,0.10,N,2.0
2,2,25.0,9600.0,MORTGAGE,1.0,MEDICAL,C,5500,12.87,0.57,N,3.0
3,3,23.0,65500.0,RENT,4.0,MEDICAL,C,35000,15.23,0.53,N,2.0
4,4,24.0,54400.0,RENT,8.0,MEDICAL,C,35000,14.27,0.55,Y,4.0


In [6]:
inputs=df.drop(['loan_grade'],axis=1)

In [7]:
target=df['loan_grade']

In [8]:
inputs

,Unnamed: 0,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,0,22.0,59000.0,RENT,14.5,PERSONAL,35000,16.02,0.59,Y,3.0
1,1,21.0,9600.0,OWN,5.0,EDUCATION,1000,11.14,0.10,N,2.0
2,2,25.0,9600.0,MORTGAGE,1.0,MEDICAL,5500,12.87,0.57,N,3.0
3,3,23.0,65500.0,RENT,4.0,MEDICAL,35000,15.23,0.53,N,2.0
4,4,24.0,54400.0,RENT,8.0,MEDICAL,35000,14.27,0.55,Y,4.0
...,...,...,...,...,...,...,...,...,...,...,...
32411,32576,40.5,53000.0,MORTGAGE,1.0,PERSONAL,5800,13.16,0.11,N,15.5
32412,32577,40.5,120000.0,MORTGAGE,4.0,PERSONAL,17625,7.49,0.15,N,15.5
32413,32578,40.5,76000.0,RENT,3.0,HOMEIMPROVEMENT,35000,10.99,0.46,N,15.5
32414,32579,40.5,140232.0,MORTGAGE,5.0,PERSONAL,15000,11.48,0.10,N,15.5


In [9]:
target

0        D
1        B
2        C
3        C
4        C
        ..
32411    C
32412    A
32413    B
32414    B
32415    B
Name: loan_grade, Length: 32416, dtype: object

In [10]:
loans_encoded = pd.get_dummies(df, columns=['person_home_ownership','loan_intent','cb_person_default_on_file'])
X = loans_encoded.drop(columns='loan_grade')
y = df[['loan_grade']]

In [11]:
X.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,32416.0,16270.898476,9417.657932,0.00,8103.75,16226.50,24433.25,32580.00
person_age,32416.0,27.451922,5.274139,20.00,23.00,26.00,30.00,40.50
person_income,32416.0,62437.029831,31807.394422,4000.00,38542.00,55000.00,79218.00,140232.00
person_emp_length,32416.0,4.682718,3.719366,0.00,2.00,4.00,7.00,14.50
loan_amnt,32416.0,9593.845632,6322.730241,500.00,5000.00,8000.00,12250.00,35000.00
loan_int_rate,32416.0,11.014662,3.083050,5.42,8.49,10.99,13.11,23.22
loan_percent_income,32416.0,0.170250,0.106812,0.00,0.09,0.15,0.23,0.83
cb_person_cred_hist_length,32416.0,5.712441,3.713420,2.00,3.00,4.00,8.00,15.50
person_home_ownership_MORTGAGE,32416.0,0.412420,0.492278,0.00,0.00,0.00,1.00,1.00
person_home_ownership_OTHER,32416.0,0.003270,0.057091,0.00,0.00,0.00,0.00,1.00


In [12]:
y['loan_grade'].value_counts()

A    10703
B    10387
C     6438
D     3620
E      963
F      241
G       64
Name: loan_grade, dtype: int64

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [14]:
from imblearn.ensemble import BalancedRandomForestClassifier
brf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf_model.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=1)

In [15]:
from sklearn.metrics import balanced_accuracy_score
predictions = brf_model.predict(X_test)
bal_acc_score = balanced_accuracy_score(y_test, predictions)
bal_acc_score

0.6314464143533718

In [16]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, predictions)
cm

array([[2390,  311,    0,    0,    0,    1,    2],
       [  82, 2412,   11,    0,    0,    5,    8],
       [   2,  218, 1152,  194,   28,   30,    5],
       [   1,   51,  171,  426,  191,   65,    7],
       [   0,   12,   13,   44,  116,   48,   17],
       [   0,    3,    3,    2,   17,   26,   21],
       [   0,    1,    1,    0,    1,    5,   11]])

In [17]:
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

          A       0.97      0.88      0.98      0.92      0.93      0.86      2704
          B       0.80      0.96      0.89      0.87      0.93      0.86      2518
          C       0.85      0.71      0.97      0.77      0.83      0.67      1629
          D       0.64      0.47      0.97      0.54      0.67      0.43       912
          E       0.33      0.46      0.97      0.38      0.67      0.43       250
          F       0.14      0.36      0.98      0.21      0.60      0.33        72
          G       0.15      0.58      0.99      0.24      0.76      0.55        19

avg / total       0.83      0.81      0.95      0.81      0.87      0.75      8104



In [18]:
brf_model.feature_importances_

array([0.08371728, 0.06100666, 0.08084584, 0.06809971, 0.0928711 ,
       0.30292494, 0.07905687, 0.06034132, 0.01498897, 0.00107866,
       0.00672137, 0.01492341, 0.01236943, 0.01553203, 0.01099662,
       0.01576679, 0.01247881, 0.01747415, 0.02398995, 0.02481609])

In [19]:
for (a,b) in sorted(zip(brf_model.feature_importances_, X.columns), reverse=True):
    print(f'{b}: ({a})')

loan_int_rate: (0.30292493808854054)
loan_amnt: (0.09287109659422217)
Unnamed: 0: (0.08371728248850108)
person_income: (0.08084584348045194)
loan_percent_income: (0.0790568701356499)
person_emp_length: (0.06809970636222562)
person_age: (0.061006658607165254)
cb_person_cred_hist_length: (0.06034131829578522)
cb_person_default_on_file_Y: (0.024816085458705783)
cb_person_default_on_file_N: (0.023989946820518308)
loan_intent_VENTURE: (0.017474154925516825)
loan_intent_MEDICAL: (0.015766793136670457)
loan_intent_EDUCATION: (0.01553203409125133)
person_home_ownership_MORTGAGE: (0.014988971131539994)
person_home_ownership_RENT: (0.014923411806479579)
loan_intent_PERSONAL: (0.01247881287566183)
loan_intent_DEBTCONSOLIDATION: (0.012369429892390699)
loan_intent_HOMEIMPROVEMENT: (0.010996617667278238)
person_home_ownership_OWN: (0.006721372240853184)
person_home_ownership_OTHER: (0.0010786559005918678)


In [20]:
brf_model.predict([[1,21,9600,5,1000,11.14,0.10,2,0,0,1,0,0,1,0,0,0,0,1,0]])	

array(['B'], dtype=object)

In [22]:
brf_model.predict([[1,40,700000,20,10000,8.02,0.014,3,0,0,1,0,0,0,0,0,1,0,1,0]])

array(['A'], dtype=object)